In [7]:
import geopandas as gpd
import pandas as pd
import json 
import psycopg2 

In [17]:
#get shark attacks geojson from inputted coordinates
   #query that gives a Feature Collection object
query_shark_attacks = '''SELECT jsonb_build_object(
      'type',     'FeatureCollection',
      'features', jsonb_agg(features.feature)
   )
   FROM (
   SELECT jsonb_build_object(
      'type',       'Feature',
      'id',          id,
      'geometry',   ST_AsGeoJSON(geometry)::jsonb,
      'properties', to_jsonb(inputs) - 'id' - 'geometry'
   ) AS feature
   FROM (SELECT 
      s."Date", s."Time", s."Location", s."Location_attack", s."Location_attack2", s.full_location, s."Location_attack3", s."Activity", s."Shark", s.geometry, s.id
      FROM shark_attacks as s
      WHERE st_intersects(s.geometry, 
                  st_transform(
                     st_buffer(
                        st_transform(
                           st_setsrid(
                              st_makepoint(-158.073760, 21.616998),4326),26904),20000),4326)
                  )
   ) inputs) features'''

In [19]:
connection = psycopg2.connect(database="geotech_ocean_haz", user="postgres", password = "postgres")

In [20]:
 #Reads the query and store it in a dataframe
feature_collection_shark_attacks = pd.read_sql(query_shark_attacks, connection)

C:\Users\johnk\anaconda3\envs\ocean_haz\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [21]:
   #Getting geojson dictionary by calling iloc[0] on the jsonb_build_object column
   feature_collection_dict_shark_attacks = feature_collection_shark_attacks.iloc[0]['jsonb_build_object']

In [22]:
 #Converting to geojson
shark_feature_collection = json.dumps(feature_collection_dict_shark_attacks)
print(shark_feature_collection)

{"type": "FeatureCollection", "features": [{"id": 4, "type": "Feature", "geometry": {"type": "Point", "coordinates": [-158.036483727, 21.48343645]}, "properties": {"Date": "03/05/2021", "Time": " 3:30 pm", "Shark": "Requiem shark, length 5-6 feet", "Activity": "Surfing", "Location": "Oahu", "full_location": "Oahu, Kaneohe", "Location_attack": " Kaneohe", "Location_attack2": " North Beach", "Location_attack3": " approx 10 yds from shore"}}, {"id": 17, "type": "Feature", "geometry": {"type": "Point", "coordinates": [-158.036483727, 21.48343645]}, "properties": {"Date": "18/09/2019", "Time": " 10:10 am", "Shark": "Galapagos shark, length 5-9 feet.", "Activity": "Swimming with sharks", "Location": "Oahu", "full_location": "Oahu, Haleiwa", "Location_attack": " Haleiwa", "Location_attack2": " approx 3 miles from shore", "Location_attack3": null}}, {"id": 18, "type": "Feature", "geometry": {"type": "Point", "coordinates": [-158.036483727, 21.48343645]}, "properties": {"Date": "18/09/2019", "T

In [24]:
#testing coral reefs 
query_coral_reefs = f'''SELECT jsonb_build_object(
      'type',     'FeatureCollection',
      'features', jsonb_agg(features.feature)
   )
   FROM (
   SELECT jsonb_build_object(
      'type',       'Feature',
      'id',         objectid,
      'geometry',   ST_AsGeoJSON(geometry)::jsonb,
      'properties', to_jsonb(inputs) - 'objectid' - 'geometry'
   ) AS feature
   FROM (SELECT 
      c.objectid, c.acres, c.geometry
      FROM coral_reefs as c
      WHERE st_intersects(c.geometry, 
                  st_transform(
                     st_buffer(
                        st_transform(
                           st_setsrid(
                              st_makepoint(-158.073760, 21.616998),4326),26904),10000),4326)
                  )
   ) inputs) features'''

In [25]:
 #Reads the query and store it in a dataframe
feature_collection_coral_reefs = pd.read_sql(query_coral_reefs, connection)

C:\Users\johnk\anaconda3\envs\ocean_haz\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [26]:
#Getting geojson dictionary by calling iloc[0] on the jsonb_build_object column
feature_collection_dict_coral_reefs = feature_collection_coral_reefs.iloc[0]['jsonb_build_object']

In [27]:
 #Converting to geojson
coral_feature_collection = json.dumps(feature_collection_dict_coral_reefs)
print(coral_feature_collection)

{"type": "FeatureCollection", "features": [{"id": 169, "type": "Feature", "geometry": {"type": "MultiPolygon", "coordinates": [[[[-158.163017262, 21.580549257], [-158.163007947, 21.580550503], [-158.162985688, 21.580547426], [-158.162962147, 21.580548823], [-158.162938609, 21.580553489], [-158.162915066, 21.580554886], [-158.162893734, 21.580557313], [-158.162879573, 21.580559304], [-158.162854194, 21.580562337], [-158.162823102, 21.580555827], [-158.162806176, 21.580551197], [-158.162777475, 21.580544255], [-158.16274676, 21.580546088], [-158.162714015, 21.580541214], [-158.162669863, 21.580533426], [-158.162627549, 21.580524173], [-158.162594257, 21.580523686], [-158.162567036, 21.580524398], [-158.162552318, 21.580520971], [-158.162540722, 21.580513669], [-158.162508525, 21.58050501], [-158.162479644, 21.580501853], [-158.162449288, 21.58049431], [-158.1624274, 21.580495103], [-158.162413241, 21.580497696], [-158.162389696, 21.580496254], [-158.162360266, 21.580494818], [-158.162338

In [30]:
query_hazard_areas = f'''SELECT jsonb_build_object(
      'type',     'FeatureCollection',
      'features', jsonb_agg(features.feature)
   )
   FROM (
   SELECT jsonb_build_object(
      'type',       'Feature',
      'id',         objectid,
      'geometry',   ST_AsGeoJSON(geometry)::jsonb,
      'properties', to_jsonb(inputs) - 'objectid' - 'geometry'
   ) AS feature
   FROM (SELECT 
      h.objectid, h.gridcode, h.geometry
      FROM hazard_areas as h
      WHERE st_intersects(h.geometry, 
                  st_transform(
                     st_buffer(
                        st_transform(
                           st_setsrid(
                              st_makepoint(-158.073760, 21.616998),4326),26904),10000),4326)
                  )
   ) inputs) features'''

In [31]:
#Reads the query and store it in a dataframe
feature_collection_hazard_areas = pd.read_sql(query_hazard_areas, connection)

DatabaseError: Execution failed on sql 'SELECT jsonb_build_object(
      'type',     'FeatureCollection',
      'features', jsonb_agg(features.feature)
   )
   FROM (
   SELECT jsonb_build_object(
      'type',       'Feature',
      'id',         objectid,
      'geometry',   ST_AsGeoJSON(geometry)::jsonb,
      'properties', to_jsonb(inputs) - 'objectid' - 'geometry'
   ) AS feature
   FROM (SELECT 
      h.objectid, h.gridcode, h.geometry
      FROM hazard_areas as h
      WHERE st_intersects(h.geometry, 
                  st_transform(
                     st_buffer(
                        st_transform(
                           st_setsrid(
                              st_makepoint(-158.073760, 21.616998),4326),26904),10000),4326)
                  )
   ) inputs) features': column h.objectid does not exist
LINE 13:       h.objectid, h.gridcode, h.geometry
               ^


In [ ]:
#Getting geojson dictionary by calling iloc[0] on the jsonb_build_object column
feature_collection_dict_hazard_areas = feature_collection_hazard_areas.iloc[0]['jsonb_build_object']

In [ ]:
 #Converting to geojson
hazard_feature_collection = json.dumps(feature_collection_dict_hazard_areaas)
print(hazard_feature_collection)